##### Copyright 2024 Google LLC.

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Reshape, UpSampling2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Data Preprocessing & Augmentation
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data = data_gen.flow_from_directory('dataset/', target_size=(150, 150), batch_size=16, class_mode='binary', subset='training')
val_data = data_gen.flow_from_directory('dataset/', target_size=(150, 150), batch_size=16, class_mode='binary', subset='validation')

# CNN-Based Deepfake Classifier
def build_cnn_classifier():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn_classifier()
cnn_model.fit(train_data, validation_data=val_data, epochs=20)

# GAN Model for Deepfake Identification
def build_generator():
    model = Sequential([
        Dense(128 * 37 * 37, activation='relu', input_shape=(100,)),
        Reshape((37, 37, 128)),
        UpSampling2D(),
        Conv2D(128, (3,3), padding='same', activation='relu'),
        UpSampling2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(3, (3,3), padding='same', activation='tanh')
    ])
    return model

def build_discriminator():
    model = Sequential([
        Conv2D(64, (3,3), padding='same', input_shape=(150, 150, 3)),
        LeakyReLU(alpha=0.2),
        MaxPooling2D(),
        Conv2D(128, (3,3), padding='same'),
        LeakyReLU(alpha=0.2),
        MaxPooling2D(),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    return model

generator = build_generator()
discriminator = build_discriminator()

discriminator.trainable = False
z = tf.keras.Input(shape=(100,))
gen_img = generator(z)
validity = discriminator(gen_img)
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training Loop for GAN
for epoch in range(20):
    noise = np.random.normal(0, 1, (16, 100))
    gen_images = generator.predict(noise)
    real_images, _ = train_data.next()
    d_loss_real = discriminator.train_on_batch(real_images, np.ones((16, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_images, np.zeros((16, 1)))
    noise = np.random.normal(0, 1, (16, 100))
    g_loss = gan.train_on_batch(noise, np.ones((16, 1)))
    print(f"Epoch {epoch+1}/20 | D Loss: {d_loss_real[0]} | G Loss: {g_loss}")

# Predict Function
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = cnn_model.predict(img_array)
    return "Fake" if prediction < 0.5 else "Real"

# Example usage:
print(predict_image('sample_image.jpg'))

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/'

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API Python quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/quickstart_colab"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/quickstart_colab.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This tutorial shows you how to get started with the Gemini API using the Python SDK.

## Prerequisites

You can run this tutorial in Google Colab, which doesn't require additional environment configuration.

Alternatively, to complete this quickstart locally, see the Python guidance in [Get started with the Gemini API](https://ai.google.dev/tutorials/quickstart).

## Install the SDK

The Python SDK for the Gemini API is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [ ]:
!pip install -q -U google-generativeai

## Set up your API key

To use the Gemini API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

<a class="button" href="https://aistudio.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Initialize the Generative Model

Before you can make any API calls, you need to initialize the Generative Model.

In [ ]:
model = genai.GenerativeModel('gemini-pro')

## Generate text

In [ ]:
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

## What's next

To learn more about working with the Gemini API, see the [Python tutorial](https://ai.google.dev/tutorials/python_quickstart).

If you're new to generative AI models, you might want to look at the
[concepts guide](https://ai.google.dev/docs/concepts) and the
[Gemini API overview](https://ai.google.dev/docs/gemini_api_overview).